# MULTI-AGENT SYSTEM (IMPROVED)

This system is similar to the code_executor_chat.
The improvement will be on making tools available and adding two other agents:
- recommender; another LLM agent that needs to answer the user. In order to do that it can call other agents (coder and a function caller).
- function caller; This is a simple agent that can only call tools to retrieve data from, for example, wikipedia.


In [1]:
from autogen import AssistantAgent, GroupChatManager, GroupChat
from autogen import register_function, config_list_from_json
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent, TEXT_FORMATS
from pathlib import Path
import os

from CypherExecutor import CypherCodeExecutor
from tools.neo4j_tools import save_schema
from tools.tools import wiki_search

In [2]:
# Save database schema and then store the path to the directory
save_schema()
p = Path(__name__).parent.resolve() / "schemas"


#### LLM MODEL CONFIGURATION #####

config_list = config_list_from_json(env_or_file="CONFIG_LIST", filter_dict={"model": "gpt-4o-mini"})
config_list[0]["api_key"] = os.environ.get("GITHUB_TOKEN")

llm_config  = {"config_list": config_list, "temperature": 0.2}

c_prompt = """
You are a data scientist.

### RULES ###
    - Use your knowledge to generate cypher queries to retrieve data from the database.
    - If the topic of the conversation is related to the database schema you need to stay silent.
    - When generating a query you must use Cypher format.
    - Format results as a table.
    - Stay silent if you won't create a query or if you won't format the results.

### DATABASE SCHEMA ### 
{input_context}
"""

r_prompt = """
You are a helpful AI assistant.

### RULES ### 
  - If the topic needs extra information or data you have to ask coder assistant 
    and use the provided tools. 
  - Reply 'TERMINATE' if messages are repeating or the task is complete.

"""


In [39]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" in x["content"][-10:].upper()

recommender   = AssistantAgent(
    "recommender",
    llm_config=llm_config,  # Turn off LLM for this agent.
    is_termination_msg=termination_msg,
    system_message=r_prompt, 
    description="Assistant that gives tasks") 

doc_retriever = RetrieveUserProxyAgent(
    name="doc_retriever",
    is_termination_msg=termination_msg,
    max_consecutive_auto_reply=3,
    human_input_mode="NEVER",
    retrieve_config={
        "docs_path": str(p), # A list of urls, dirs or files can be passed
        "extra-docs": True,
        "model": config_list[0]["model"],
        "get_or_create": True,
        "customized_prompt": """CONTEXT:\n{input_context}\n\nQUESTION:\n{input_question}"""
    },
    code_execution_config=False,
    description="Assistant who has extra content retrieval power."
)

# Coder generates cypher queries with the help of the db schema 
coder      = AssistantAgent(
    name="coder",
    is_termination_msg=termination_msg,
    system_message=c_prompt,
    llm_config=llm_config,
    description="Data scientist that creates cypher queries."
)

# Executor runs the query in a jupyter notebook
executor   = AssistantAgent(
    name="executor",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER", # THIS IS NOT SECURE! 
    description= "Executes queries.",
    code_execution_config={"executor": CypherCodeExecutor()}
    )

# Fun_caller calls a tool 
fun_caller = AssistantAgent(
    "fun_caller",
    llm_config=False,  # Turn off LLM for this agent.
    is_termination_msg=termination_msg,
    description="Executes tools.")


#### REGISTER FUNCTIONS ####

register_function(wiki_search,
                  caller= recommender,
                  executor= fun_caller,
                  description="Service that will retrieve information about 1 or 2 words.")

#############################

 
PROBLEM  = "Can you reccomend me a movie directed by Paul w.s. anderson?"

def _reset_agents():
    """
    This function reset all the agents used for the group chat.
    This should be used every time you start a new conversation.
    """
    doc_retriever.reset()
    coder.reset()
    executor.reset()
    recommender.reset()

def state_transition(last_speaker, groupchat):
    """
    This function simply return the next speaker based on the last one.
    In this case we decided that doc_retriever should speak only to initiate chat
    and then the chat is from coder to executor in a sort of loop.
    """

    m = groupchat.messages[-1]
    
    if last_speaker in [fun_caller, doc_retriever]: return recommender
    elif last_speaker is executor: return coder 
    elif last_speaker is recommender: 
        if "tool_calls" in m: return fun_caller
        else: return coder
    
    elif last_speaker is coder:
        if "```cypher" in m["content"]: return executor
        else: return recommender
        
def rag_chat():
    """
    Resets the agents, creates the group chat and then initiate chat.
    The first agent to speak is the doc_retriever that will only pass:
        - prompt
        - files (like the schema), that is the context
        - question
    """
    _reset_agents()
    groupchat = GroupChat(agents=[doc_retriever, recommender, coder, fun_caller, executor], messages=[], 
                          max_round=15, speaker_selection_method=state_transition)
    manager   = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    doc_retriever.initiate_chat(manager,message=doc_retriever.message_generator, problem=PROBLEM,n_results=3)


In [4]:
#### START OF THE GROUP CHAT ####
rag_chat()

2024-10-03 10:33:39,678 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `autogen-docs`.
2024-10-03 10:33:39,695 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 1 chunks.


Trying to create collection.
VectorDB returns doc_ids:  [['ca5f356a', 'c99bc2e0', '985292cd']]
Adding content of doc ca5f356a to context.
Adding content of doc c99bc2e0 to context.
Adding content of doc 985292cd to context.
doc_retriever (to chat_manager):

CONTEXT:
[
    {
        "nodes": [
            {
                "name": "Movie",
                "indexes": [],
                "constraints": [],
                "properties": [
                    "tagline",
                    "title",
                    "released"
                ]
            },
            {
                "name": "Person",
                "indexes": [],
                "constraints": [],
                "properties": [
                    "born",
                    "name"
                ]
            }
        ],
        "relationships": [
            {
                "name": "ACTED_IN",
                "node1": "Person",
                "node2": "Movie",
                "properties": [
               

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: released)} {position: line: 2, column: 34, offset: 102} for query: 'MATCH (p:Person {name: "Paul W.S. Anderson"})-[:DIRECTED]->(m:Movie)\nRETURN m.title AS Movie_Title, m.released AS Release_Year, m.tagline AS Tagline\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that

Neo4jError: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: released)} {position: line: 2, column: 34, offset: 102}

executor (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
{severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: released)} {position: line: 2, column: 34, offset: 102}

--------------------------------------------------------------------------------

Next speaker: coder

coder (to chat_manager):

```cypher
MATCH (p:Person {name: "Paul W.S. Anderson"})-[:DIRECTED]->(m:Movie)
RETURN m.title AS Movie_Title, m.tagline AS Tagline
```

--------------------------------------------------------------------------------

Next speaker: executor


>>>>>>>> EXECUTING CODE BLOCK (inferred language is cypher)...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: tagline)} {position: line: 2, column: 34, offset: 102} for query: 'MATCH (p:Person {name: "Paul W.S. Anderson"})-[:DIRECTED]->(m:Movie)\nRETURN m.title AS Movie_Title, m.tagline AS Tagline\n'


Neo4jError: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: tagline)} {position: line: 2, column: 34, offset: 102}

executor (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
{severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: tagline)} {position: line: 2, column: 34, offset: 102}

--------------------------------------------------------------------------------

Next speaker: coder

coder (to chat_manager):

```cypher
MATCH (p:Person {name: "Paul W.S. Anderson"})-[:DIRECTED]->(m:Movie)
RETURN m.title AS Movie_Title
```

--------------------------------------------------------------------------------

Next speaker: executor


>>>>>>>> EXECUTING CODE BLOCK (inferred language is cypher)...


[{'Movie_Title': 'Mortal Kombat'}, {'Movie_Title': 'Shopping'}]

executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: [{'Movie_Title': 'Mortal Kombat'}, {'Movie_Title': 'Shopping'}]

--------------------------------------------------------------------------------

Next speaker: coder

coder (to chat_manager):

| Movie_Title      |
|-------------------|
| Mortal Kombat     |
| Shopping          |

--------------------------------------------------------------------------------

Next speaker: recommender

recommender (to chat_manager):

Here are two movies directed by Paul W.S. Anderson:

1. **Mortal Kombat**
2. **Shopping**

If you need more information or further recommendations, feel free to ask!

--------------------------------------------------------------------------------

Next speaker: coder

coder (to chat_manager):



--------------------------------------------------------------------------------

Next speaker: recommender

recommender (to chat_manager):

TERMINATE

---------------------------------------------------